# 머신러닝을 위한 확률과 통계 기초 개념 정리

## 학습 목표
- 확률의 정의와 조건부 확률 완벽 이해
- 확률변수, 기댓값, 분산 등 통계량 마스터
- 주요 확률분포들의 특성과 ML 활용
- 베이즈 정리와 중심극한정리의 직관적 이해
- Python 코드를 통한 실습과 시각화

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# 설정
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.family'] = 'DejaVu Sans'
np.random.seed(42)

## 1. 확률의 정의와 기본 공리

### 1.1 확률이란?

**확률(Probability)**은 0과 1 사이의 값으로, 어떤 사건이 발생할 가능성의 정도를 나타냅니다.

### 1.2 Kolmogorov 공리

1. **비음성**: $P(A) \geq 0$ for all events $A$
2. **정규화**: $P(S) = 1$ (전체 표본공간의 확률은 1)
3. **가산 가법성**: 상호 배타적 사건 $A_1, A_2, \ldots$에 대해 
   $$P(A_1 \cup A_2 \cup \cdots) = P(A_1) + P(A_2) + \cdots$$

### 1.3 기본 성질

- **여사건**: $P(A^c) = 1 - P(A)$
- **합사건**: $P(A \cup B) = P(A) + P(B) - P(A \cap B)$
- **독립**: $A \perp B \iff P(A \cap B) = P(A)P(B)$

In [ ]:
# 예제: 주사위 던지기
outcomes = np.arange(1, 7)
probs = np.ones(6) / 6

plt.figure(figsize=(10, 5))
plt.bar(outcomes, probs, color='steelblue', alpha=0.7, edgecolor='black')
plt.xlabel('주사위 눈금')
plt.ylabel('확률')
plt.title('균등 분포: 공정한 주사위')
plt.ylim(0, 0.3)
for i, (x, y) in enumerate(zip(outcomes, probs)):
    plt.text(x, y + 0.01, f'1/6', ha='center')
plt.show()

print("확률 공리 검증:")
print(f"1. 모든 확률 >= 0: {np.all(probs >= 0)}")
print(f"2. 전체 확률 = 1: {np.sum(probs):.4f}")
print(f"3. 합사건: P(1 or 2) = {probs[0] + probs[1]:.4f}")

## 2. 조건부 확률과 베이즈 정리

### 2.1 조건부 확률 (Conditional Probability)

사건 $B$가 일어났다는 조건 하에 사건 $A$가 일어날 확률:

$$P(A|B) = \frac{P(A \cap B)}{P(B)}, \quad P(B) > 0$$

**직관**: 표본공간이 $B$로 축소된 상황에서 $A$의 확률을 재정규화

### 2.2 곱셈 규칙

$$P(A \cap B) = P(A|B)P(B) = P(B|A)P(A)$$

### 2.3 베이즈 정리 (Bayes' Theorem)

$$P(A|B) = \frac{P(B|A)P(A)}{P(B)}$$

- $P(A)$: **Prior** (사전 확률)
- $P(B|A)$: **Likelihood** (가능도)
- $P(B)$: **Evidence** (증거)
- $P(A|B)$: **Posterior** (사후 확률)

### 2.4 전확률 공식 (Law of Total Probability)

$$P(B) = \sum_{i=1}^{n} P(B|A_i) P(A_i)$$

**ML에서의 활용**: 나이브 베이즈 분류기, 베이지안 네트워크, 스팸 필터 등

In [ ]:
# 베이즈 정리 예제: 질병 진단
# A: 질병이 있다, B: 검사 결과 양성

P_disease = 0.01  # Prior: 질병 유병률 1%
P_positive_given_disease = 0.95  # Likelihood: 민감도
P_negative_given_no_disease = 0.90  # 특이도
P_positive_given_no_disease = 1 - P_negative_given_no_disease  # 위양성

# Evidence: 전체 양성 확률
P_positive = (P_positive_given_disease * P_disease + 
              P_positive_given_no_disease * (1 - P_disease))

# Posterior: 양성일 때 실제로 질병이 있을 확률
P_disease_given_positive = (P_positive_given_disease * P_disease) / P_positive

print("=== 베이즈 정리: 질병 진단 ===")
print(f"Prior P(질병) = {P_disease:.4f}")
print(f"Likelihood P(양성|질병) = {P_positive_given_disease:.4f}")
print(f"위양성 P(양성|건강) = {P_positive_given_no_disease:.4f}")
print(f"Evidence P(양성) = {P_positive:.4f}")
print(f"\nPosterior P(질병|양성) = {P_disease_given_positive:.4f}")
print(f"\n해석: 검사 결과가 양성이어도 실제 질병일 확률은 {P_disease_given_positive*100:.1f}%")
print("→ 낮은 유병률 때문! (Base rate fallacy)")

# 시각화
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
labels = ['질병 있음', '질병 없음']
colors = ['#ff6b6b', '#51cf66']

# Prior
ax[0].pie([P_disease, 1 - P_disease], labels=labels, colors=colors, autopct='%1.2f%%')
ax[0].set_title('Prior: 검사 전 확률')

# Posterior
ax[1].pie([P_disease_given_positive, 1 - P_disease_given_positive], 
          labels=labels, colors=colors, autopct='%1.2f%%')
ax[1].set_title('Posterior: 양성 결과 후 확률')

plt.tight_layout()
plt.show()

## 3. 확률변수 (Random Variable)

### 3.1 정의

**확률변수**는 표본공간의 각 결과를 수치로 매핑하는 함수 $X: \Omega \rightarrow \mathbb{R}$

### 3.2 이산형 vs 연속형

**이산 확률변수 (Discrete)**:
- 가산개의 값 (예: 동전 던지기, 주사위)
- **PMF (Probability Mass Function)**: $p(x) = P(X = x)$
- 성질: $\sum_x p(x) = 1$

**연속 확률변수 (Continuous)**:
- 연속 범위의 값 (예: 키, 몸무게)
- **PDF (Probability Density Function)**: $f(x)$
- 성질: $\int_{-\infty}^{\infty} f(x) dx = 1$
- $P(a \leq X \leq b) = \int_a^b f(x) dx$
- 중요: $P(X = a) = 0$ (특정 값의 확률은 0!)

**ML에서의 활용**: 모델 예측값, 손실 함수, 가중치 초기화 등

In [ ]:
# 이산형: 이항 분포 vs 연속형: 정규 분포
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 이산형: Binomial
n, p = 10, 0.5
x_discrete = np.arange(0, n+1)
pmf = stats.binom.pmf(x_discrete, n, p)

axes[0].bar(x_discrete, pmf, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].set_xlabel('X (성공 횟수)')
axes[0].set_ylabel('P(X = x)')
axes[0].set_title(f'이산형: Binomial(n={n}, p={p})')
axes[0].grid(alpha=0.3)

# 연속형: Normal
mu, sigma = 0, 1
x_continuous = np.linspace(-4, 4, 1000)
pdf = stats.norm.pdf(x_continuous, mu, sigma)

axes[1].plot(x_continuous, pdf, 'r-', linewidth=2)
axes[1].fill_between(x_continuous, pdf, alpha=0.3, color='red')
axes[1].set_xlabel('X')
axes[1].set_ylabel('f(x)')
axes[1].set_title(f'연속형: Normal(μ={mu}, σ²={sigma**2})')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("=== 확률 계산 ===")
print(f"이산형: P(X = 5) = {pmf[5]:.4f}")
prob_continuous = stats.norm.cdf(1, mu, sigma) - stats.norm.cdf(-1, mu, sigma)
print(f"연속형: P(-1 ≤ X ≤ 1) = {prob_continuous:.4f} (약 68%, 정규분포의 1σ 구간)")

## 4. 기댓값, 분산, 공분산, 상관계수

### 4.1 기댓값 (Expectation, Mean)

확률분포의 **평균적인 값**, 가중평균:

**이산형**: $E[X] = \sum_x x \cdot P(X = x)$

**연속형**: $E[X] = \int_{-\infty}^{\infty} x f(x) dx$

**직관**: 무한히 반복 관측했을 때의 평균 (대수의 법칙)

### 4.2 분산 (Variance)과 표준편차

평균 주변 **산포도**:

$$\text{Var}(X) = E[(X - E[X])^2] = E[X^2] - (E[X])^2$$

**표준편차**: $\sigma(X) = \sqrt{\text{Var}(X)}$

### 4.3 공분산 (Covariance)

두 변수의 **함께 변동하는 정도**:

$$\text{Cov}(X, Y) = E[(X - E[X])(Y - E[Y])]$$

- 양수: 함께 증가
- 음수: 반대로 움직임
- 0: 선형 관계 없음

### 4.4 상관계수 (Correlation Coefficient)

공분산을 **정규화**:

$$\rho_{X,Y} = \frac{\text{Cov}(X, Y)}{\sigma(X) \sigma(Y)}, \quad -1 \leq \rho \leq 1$$

**ML에서의 활용**: 특성 선택, PCA, 다중공선성 제거, 협업 필터링 등

In [ ]:
# 대수의 법칙 (Law of Large Numbers) 시뮬레이션
# 동전 던지기 1000번
n_trials = 1000
p = 0.5  # 앞면 확률

flips = np.random.binomial(1, p, n_trials)
cumulative_avg = np.cumsum(flips) / (np.arange(n_trials) + 1)

plt.figure(figsize=(12, 5))
plt.plot(cumulative_avg, linewidth=2, label='누적 평균 (앞면 비율)')
plt.axhline(p, color='r', linestyle='--', linewidth=2, label=f'이론적 확률 p={p}')
plt.xlabel('시행 횟수')
plt.ylabel('앞면의 누적 비율')
plt.title('대수의 법칙: 동전 던지기 실험')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

print(f"1000번 후 누적 평균: {cumulative_avg[-1]:.4f}")
print(f"이론적 기댓값: {p}")

In [ ]:
# 공분산과 상관계수 시각화
mean = [0, 0]
cov_high = [[1, 0.8], [0.8, 1]]  # 높은 상관관계
cov_zero = [[1, 0], [0, 1]]  # 독립

samples_corr = np.random.multivariate_normal(mean, cov_high, 1000)
samples_indep = np.random.multivariate_normal(mean, cov_zero, 1000)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 상관관계 있음
axes[0].scatter(samples_corr[:, 0], samples_corr[:, 1], alpha=0.5, s=10)
axes[0].set_xlabel('X')
axes[0].set_ylabel('Y')
axes[0].set_title('높은 상관관계 (ρ = 0.8)')
axes[0].grid(alpha=0.3)
axes[0].axis('equal')

# 독립
axes[1].scatter(samples_indep[:, 0], samples_indep[:, 1], alpha=0.5, s=10, color='orange')
axes[1].set_xlabel('X')
axes[1].set_ylabel('Y')
axes[1].set_title('독립 (ρ = 0)')
axes[1].grid(alpha=0.3)
axes[1].axis('equal')

plt.tight_layout()
plt.show()

# 상관계수 계산
corr_high = np.corrcoef(samples_corr.T)[0, 1]
corr_zero = np.corrcoef(samples_indep.T)[0, 1]
print(f"높은 상관: ρ = {corr_high:.4f}")
print(f"독립: ρ = {corr_zero:.4f}")

## 5. 주요 확률분포

### 5.1 베르누이 분포 (Bernoulli)

**설명**: 1번 시행, 성공(1)/실패(0)

- $P(X = 1) = p$, $P(X = 0) = 1 - p$
- $E[X] = p$
- $\text{Var}(X) = p(1-p)$

**ML 활용**: 이진 분류 레이블, 로지스틱 회귀, 드롭아웃

In [ ]:
# 베르누이 분포 시뮬레이션
p = 0.6
n_samples = 10000
samples = np.random.binomial(1, p, n_samples)

print(f"=== Bernoulli(p={p}) ===")
print(f"이론 E[X] = {p}")
print(f"이론 Var(X) = {p*(1-p):.4f}")
print(f"샘플 평균 = {samples.mean():.4f}")
print(f"샘플 분산 = {samples.var():.4f}")

### 5.2 이항분포 (Binomial)

**설명**: $n$번 독립 시행에서 성공 횟수

$$P(X = k) = \binom{n}{k} p^k (1-p)^{n-k}$$

- $E[X] = np$
- $\text{Var}(X) = np(1-p)$

**ML 활용**: 베이지안 추론, A/B 테스트, 정확도 분포 근사

In [ ]:
# 이항분포 시각화
n, p = 20, 0.7
k_values = np.arange(0, n+1)
pmf = stats.binom.pmf(k_values, n, p)

plt.figure(figsize=(10, 6))
plt.bar(k_values, pmf, alpha=0.7, color='orange', edgecolor='black')
plt.axvline(n*p, color='r', linestyle='--', linewidth=2, label=f'E[X] = {n*p}')
plt.xlabel('k (성공 횟수)')
plt.ylabel('P(X = k)')
plt.title(f'Binomial Distribution: n={n}, p={p}')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

### 5.3 범주형 분포 (Categorical)

**설명**: $K$개 범주 중 하나 선택

- $P(Y = i) = p_i$, $\sum_{i=1}^K p_i = 1$
- $K=2$일 때 베르누이와 동일

**ML 활용**: 다항 분류 출력(Softmax), 언어 모델, 강화학습 행동 선택

### 5.4 포아송 분포 (Poisson)

**설명**: 단위 시간/공간당 사건 발생 횟수

$$P(X = k) = \frac{\lambda^k e^{-\lambda}}{k!}$$

- $E[X] = \text{Var}(X) = \lambda$

**ML 활용**: 포아송 회귀, 카운트 데이터 모델링, 드문 이벤트 예측

In [ ]:
# 포아송 분포 비교
lambdas = [1, 5, 10]
k_values = np.arange(0, 25)

plt.figure(figsize=(12, 6))
for lam in lambdas:
    pmf = stats.poisson.pmf(k_values, lam)
    plt.plot(k_values, pmf, 'o-', label=f'λ={lam}, E[X]={lam}', markersize=5)

plt.xlabel('k (발생 횟수)')
plt.ylabel('P(X = k)')
plt.title('Poisson Distribution for Different λ')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

### 5.5 지수분포 (Exponential)

**설명**: 사건 발생까지 대기 시간 (연속형)

$$f(x) = \lambda e^{-\lambda x}, \quad x \geq 0$$

- $E[X] = \frac{1}{\lambda}$
- $\text{Var}(X) = \frac{1}{\lambda^2}$
- **무기억성**: $P(X > s+t | X > s) = P(X > t)$

**ML 활용**: 생존 분석, 신뢰성 공학, 포아송 프로세스의 간격 분포

In [ ]:
# 지수분포 vs 감마분포
x = np.linspace(0, 10, 1000)

plt.figure(figsize=(12, 6))

# 지수분포 (감마의 특수 케이스: α=1)
lam = 1
exp_pdf = stats.expon.pdf(x, scale=1/lam)
plt.plot(x, exp_pdf, linewidth=2, label=f'Exponential(λ={lam})')

# 감마분포 (여러 지수분포의 합)
for alpha in [2, 3, 5]:
    gamma_pdf = stats.gamma.pdf(x, alpha, scale=1/lam)
    plt.plot(x, gamma_pdf, linewidth=2, label=f'Gamma(α={alpha}, λ={lam})')

plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('Exponential vs Gamma Distributions')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

### 5.6 정규분포 (Normal/Gaussian)

**가장 중요한 분포!**

$$f(x) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(x-\mu)^2}{2\sigma^2}\right)$$

- $E[X] = \mu$
- $\text{Var}(X) = \sigma^2$
- 표기: $X \sim \mathcal{N}(\mu, \sigma^2)$

**68-95-99.7 규칙**:
- $P(\mu - \sigma \leq X \leq \mu + \sigma) \approx 0.68$
- $P(\mu - 2\sigma \leq X \leq \mu + 2\sigma) \approx 0.95$
- $P(\mu - 3\sigma \leq X \leq \mu + 3\sigma) \approx 0.997$

**ML 활용**: 선형회귀 오차항, 가우시안 나이브 베이즈, GMM, VAE, 가중치 초기화

In [ ]:
# 정규분포와 68-95-99.7 규칙
mu, sigma = 0, 1
x = np.linspace(-4, 4, 1000)
pdf = stats.norm.pdf(x, mu, sigma)

plt.figure(figsize=(12, 7))
plt.plot(x, pdf, 'b-', linewidth=2, label='PDF')

# 68-95-99.7 표시
for k, alpha, label in [(1, 0.3, '68%'), (2, 0.2, '95%'), (3, 0.1, '99.7%')]:
    x_range = np.linspace(mu - k*sigma, mu + k*sigma, 100)
    y_range = stats.norm.pdf(x_range, mu, sigma)
    plt.fill_between(x_range, y_range, alpha=alpha, label=f'±{k}σ ({label})')

plt.axvline(mu, color='r', linestyle='--', linewidth=2, label=f'μ = {mu}')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title(f'Normal Distribution N({mu}, {sigma}²)')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# 검증
print("68-95-99.7 규칙 검증:")
for k in [1, 2, 3]:
    prob = stats.norm.cdf(mu + k*sigma, mu, sigma) - stats.norm.cdf(mu - k*sigma, mu, sigma)
    print(f"P(μ-{k}σ ≤ X ≤ μ+{k}σ) = {prob:.4f}")

## 6. 중심극한정리 (Central Limit Theorem)

### 정리

독립 확률변수들의 **표본평균**은 표본 크기가 커질수록 정규분포로 수렴:

$$\bar{X}_n = \frac{1}{n}\sum_{i=1}^n X_i \approx \mathcal{N}\left(\mu, \frac{\sigma^2}{n}\right)$$

### 의의

- 개별 분포의 형태와 **무관**
- 신뢰구간, 가설검정의 이론적 근거
- 자연현상이 정규분포를 따르는 이유

**ML 활용**: SGD의 그래디언트 분포, 앙상블 예측 분산 감소, 오차항 가정

In [ ]:
# 중심극한정리 시뮬레이션
# 지수분포(편향 분포)의 표본평균 → 점점 정규분포로

lam = 1  # 지수분포 모수
sample_sizes = [1, 5, 30]
n_simulations = 10000

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, n in enumerate(sample_sizes):
    # n개씩 샘플링하여 평균 계산
    samples = np.random.exponential(1/lam, (n_simulations, n))
    sample_means = samples.mean(axis=1)
    
    ax = axes[idx]
    ax.hist(sample_means, bins=50, density=True, alpha=0.7, color='skyblue', edgecolor='black')
    
    # 이론적 정규분포 (CLT)
    mu_theory = 1/lam
    sigma_theory = (1/lam) / np.sqrt(n)
    x = np.linspace(sample_means.min(), sample_means.max(), 100)
    ax.plot(x, stats.norm.pdf(x, mu_theory, sigma_theory), 'r-', linewidth=2, label='이론 N(μ, σ²/n)')
    
    ax.set_title(f'표본 크기 n={n}')
    ax.set_xlabel('표본평균')
    ax.set_ylabel('밀도')
    ax.legend()
    ax.grid(alpha=0.3)

plt.suptitle('중심극한정리: 지수분포의 표본평균 분포', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("해석: 표본 크기가 커질수록 (좌→우) 분포가 정규분포에 가까워진다!")

## 7. Softmax와 확률의 연결

### Softmax 함수

Logits $z = [z_1, z_2, \ldots, z_K]$를 확률 분포로 변환:

$$\sigma(z)_i = \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}}$$

### 왜 확률인가?

1. **비음성**: $\sigma(z)_i \geq 0$ (지수함수는 항상 양수)
2. **정규화**: $\sum_{i=1}^{K} \sigma(z)_i = 1$

따라서 PMF의 공리를 만족!

### 해석

$$\sigma(z)_i = P(Y = i | X)$$

입력 $X$가 주어졌을 때 클래스 $i$일 조건부 확률 = **Categorical Distribution**

In [ ]:
def softmax(z):
    """Numerically stable softmax"""
    exp_z = np.exp(z - np.max(z))
    return exp_z / np.sum(exp_z)

# 3개 클래스 분류
logits = np.array([3.0, 1.0, 0.2])
probs = softmax(logits)

print("=== Softmax 예제 ===")
print(f"Logits: {logits}")
print(f"Softmax: {probs}")
print(f"Sum: {np.sum(probs):.10f} (should be 1.0)")
print(f"\n해석:")
for i, p in enumerate(probs):
    print(f"  P(Y = {i} | X) = {p:.4f} ({p*100:.2f}%)")

# Temperature scaling
def softmax_temperature(z, T=1.0):
    z_scaled = z / T
    exp_z = np.exp(z_scaled - np.max(z_scaled))
    return exp_z / np.sum(exp_z)

temperatures = [0.5, 1.0, 2.0, 5.0]
plt.figure(figsize=(12, 4))
x = np.arange(len(logits))
width = 0.2

for i, T in enumerate(temperatures):
    probs_T = softmax_temperature(logits, T)
    plt.bar(x + i*width, probs_T, width, label=f'T = {T}', alpha=0.8)

plt.xlabel('클래스')
plt.ylabel('확률')
plt.title('Temperature Scaling 효과')
plt.xticks(x + width * 1.5, ['Class 0', 'Class 1', 'Class 2'])
plt.legend()
plt.grid(alpha=0.3, axis='y')
plt.show()

print("\n온도 효과: T>1이면 균등, T<1이면 뾰족")

## 8. Cross-Entropy Loss와 MLE

### Cross-Entropy Loss

$$L = -\sum_{i=1}^{K} y_i \log \hat{y}_i$$

- $y$: one-hot 인코딩된 정답
- $\hat{y} = \sigma(z)$: Softmax 출력

### 정보 이론적 해석

두 분포 $p$와 $q$ 사이의 거리:

$$H(p, q) = -\sum_i p(i) \log q(i)$$

최소화하면 $q \rightarrow p$

### Maximum Likelihood Estimation (MLE)

$$\theta^* = \arg\max_{\theta} P(\text{data} | \theta)$$

**중요**: Minimize Cross-Entropy = Maximize Log-Likelihood!

In [ ]:
def cross_entropy(y_true, y_pred):
    return -np.sum(y_true * np.log(y_pred + 1e-10))

# True label: class 2
y_true = np.array([0, 0, 1])

predictions = [
    np.array([0.1, 0.1, 0.8]),  # 좋은 예측
    np.array([0.3, 0.4, 0.3]),  # 나쁜 예측
    np.array([0.0, 0.0, 1.0]),  # 완벽한 예측
]

print("=== Cross-Entropy Loss ===")
for i, y_pred in enumerate(predictions):
    loss = cross_entropy(y_true, y_pred)
    print(f"예측 {i+1}: {y_pred}, Loss = {loss:.4f}, P(정답) = {y_pred[2]:.1f}")

# Loss landscape
p_true = np.linspace(0.01, 0.99, 100)
losses = [cross_entropy(y_true, np.array([(1-p)/2, (1-p)/2, p])) for p in p_true]

plt.figure(figsize=(10, 5))
plt.plot(p_true, losses, 'b-', linewidth=2)
plt.xlabel('P(true class)')
plt.ylabel('Cross-Entropy Loss')
plt.title('Loss vs. Predicted Probability')
plt.grid(alpha=0.3)
plt.show()

print("\n해석: 정답 확률이 1에 가까울수록 loss → 0")

## 9. 연습문제

### 문제 1: 베이즈 정리

두 동전이 있습니다:
- 동전 A: 공정 (P(앞면) = 0.5)
- 동전 B: 편향 (P(앞면) = 0.7)

랜덤하게 하나를 선택하여 던졌더니 앞면이 나왔습니다.

1. P(동전 A | 앞면)을 구하시오
2. 같은 동전을 다시 던졌을 때 앞면이 나올 확률은?

In [ ]:
# 문제 1 풀이
P_A = 0.5
P_B = 0.5
P_heads_given_A = 0.5
P_heads_given_B = 0.7

# Evidence
P_heads = P_heads_given_A * P_A + P_heads_given_B * P_B

# Posterior
P_A_given_heads = (P_heads_given_A * P_A) / P_heads
P_B_given_heads = (P_heads_given_B * P_B) / P_heads

print("=== 문제 1 풀이 ===")
print(f"1. P(동전 A | 앞면) = {P_A_given_heads:.4f}")
print(f"   P(동전 B | 앞면) = {P_B_given_heads:.4f}")

# 다시 던졌을 때
P_heads_again = P_heads_given_A * P_A_given_heads + P_heads_given_B * P_B_given_heads
print(f"2. P(다시 앞면) = {P_heads_again:.4f}")
print(f"   → 처음({P_heads:.2f})보다 높아짐!")

### 문제 2: 정규분포

학생들의 시험 점수가 평균 70, 표준편차 10인 정규분포를 따릅니다.

1. 85점의 Z-score는?
2. 80점 이상 받을 확률은?
3. 상위 10%의 최소 점수는?

In [ ]:
# 문제 2 풀이
mu, sigma = 70, 10

print("=== 문제 2: N(70, 10²) ===")

# 1. Z-score
x = 85
z = (x - mu) / sigma
print(f"1. Z-score of {x}: z = {z:.2f}")

# 2. P(X ≥ 80)
prob_80 = 1 - stats.norm.cdf(80, mu, sigma)
print(f"2. P(X ≥ 80) = {prob_80:.4f}")

# 3. 상위 10%
top_10 = stats.norm.ppf(0.9, mu, sigma)
print(f"3. 상위 10% 최소 점수 = {top_10:.2f}점")

## 10. 요약 정리

### 핵심 개념

1. **확률**: 사건 발생 가능성의 척도 (0~1)
2. **조건부 확률**: $P(A|B) = \frac{P(A \cap B)}{P(B)}$
3. **베이즈 정리**: Prior × Likelihood → Posterior
4. **확률변수**: 표본공간 → 실수의 함수
5. **기댓값**: 확률분포의 평균
6. **분산**: 평균 주변 산포도
7. **중심극한정리**: 표본평균 → 정규분포

### 주요 분포

| 분포 | 타입 | 파라미터 | E[X] | Var(X) | ML 활용 |
|------|------|----------|------|--------|--------|
| Bernoulli | 이산 | p | p | p(1-p) | 이진 분류 |
| Binomial | 이산 | n, p | np | np(1-p) | A/B 테스트 |
| Poisson | 이산 | λ | λ | λ | 카운트 데이터 |
| Categorical | 이산 | p₁,...,pₖ | - | - | 다항 분류 |
| Exponential | 연속 | λ | 1/λ | 1/λ² | 생존 분석 |
| Normal | 연속 | μ, σ² | μ | σ² | **모든 곳** |

### ML에서 확률의 역할

- **모델링**: 데이터 분포 가정
- **추론**: 베이지안 방법
- **학습**: MLE, MAP
- **평가**: 불확실성 정량화
- **정규화**: Prior = Regularization

---

## 참고자료

- Introduction to Probability (Bertsekas & Tsitsiklis)
- Pattern Recognition and Machine Learning (Bishop)
- Dive into Deep Learning (d2l.ai)
- Stanford CS229 Probability Review